# Using numerical and categorical variables together

In the previous notebooks, we showed the required preprocessing to apply
when dealing with numerical and categorical variables. However, we decoupled
the process to treat each type individually. In this notebook, we will show
how to combine these preprocessing steps.

We will first load the entire adult census dataset.

In [1]:
import pandas as pd

adult_census = pd.read_pickle('../data/adult_census.pkl')

In [2]:
adult_census = adult_census.drop(columns = 'education-num')

target_name = 'class'
target = adult_census[target_name]

data = adult_census.drop(columns = [target_name])

## Selection based on data types

We will separate categorical and numerical variables using their data
types to identify them, as we saw previously that `object` corresponds
to categorical columns (strings). We make use of `make_column_selector`
helper to select the corresponding columns.

In [3]:
from sklearn.compose import make_column_selector as selector

numerical_column_selector = selector(dtype_exclude= object)
categorical_column_selector = selector(dtype_include=object)


numerical_columns = numerical_column_selector(data)
categorical_columns = categorical_column_selector(data)

<div class="admonition caution alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Caution!</p>
<p class="last">Here, we know that <tt class="docutils literal">object</tt> data type is used to represent strings and thus
categorical features. Be aware that this is not always the case. Sometimes
<tt class="docutils literal">object</tt> data type could contain other type of information (e.g. dates that
were not parsed) and you should manually introspect the content of your
dataframe to not wrongly use <tt class="docutils literal">make_column_selector</tt>.</p>
</div>

## Dispatch columns to a specific processor

In the previous sections, we saw that we need to treat data differently
depending on their nature (i.e. numerical or categorical).

Scikit-learn provides a `ColumnTransformer` class which will send specific
columns to a specific transformer, making it easy to fit a single predictive
model on a dataset that combines both kinds of variables together
(heterogeneously typed tabular data).

We first define the columns depending on their data type:

* **one-hot encoding** will be applied to categorical columns. Besides, we
  use `handle_unknown="ignore"` to solve the potential issues due to rare
  categories.
* **numerical scaling** numerical features which will be standardized.

Now, we create our `ColumnTransfomer` by specifying three values:
the preprocessor name, the transformer, and the columns.
First, let's create the preprocessors for the numerical and categorical
parts.

In [4]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [5]:
categorical_preprocessor = OneHotEncoder(handle_unknown='ignore')
numerical_preprocessor = StandardScaler()

Now, we create the transformer and associate each of these preprocessors
with their respective columns.

In [6]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)
])

A `ColumnTransformer` does the following:

* It **splits the columns** of the original dataset based on the column names
  or indices provided. We will obtain as many subsets as the number of
  transformers passed into the `ColumnTransformer`.
* It **transforms each subsets**. A specific transformer is applied to
  each subset: it will internally call `fit_transform` or `transform`. The
  output of this step is a set of transformed datasets.
* It then **concatenate the transformed datasets** into a single dataset.

The important thing is that `ColumnTransformer` is like any other
scikit-learn transformer. In particular it can be combined with a classifier
in a `Pipeline`:

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))

We can display an interactive diagram with the following command:

In [9]:
from sklearn import set_config
set_config(display = 'diagram')
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country']),
                                                 ('standard-scaler',
                                                  StandardScaler(),
                                                  ['age', 'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

The final model is more complex than the previous models but still follows
the same API (the same set of methods that can be called by the user):

- the `fit` method is called to preprocess the data and then train the
  classifier of the preprocessed data;
- the `predict` method makes predictions on new data;
- the `score` method is used to predict on the test data and compare the
  predictions to the expected test labels to compute the accuracy.

Let's start by splitting our data into train and test sets.

In [11]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(data, target, random_state=42)


<div class="admonition caution alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Caution!</p>
<p class="last">Be aware that we use <tt class="docutils literal">train_test_split</tt> here for didactic purposes, to show
the scikit-learn API.</p>
</div>

Now, we can train the model on the train set.

In [12]:
_ = model.fit(data_train, target_train)

Then, we can send the raw dataset straight to the pipeline. Indeed, we do not
need to make any manual preprocessing (calling the `transform` or
`fit_transform` methods) as it will be handled when calling the `predict`
method. As an example, we predict on the five first samples from the test
set.

In [13]:
data_test.head()

,age,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country
7762,56,0,0,40,Private,HS-grad,Divorced,Other-service,Unmarried,White,Female,United-States
23881,25,0,0,40,Private,HS-grad,Married-civ-spouse,Transport-moving,Own-child,Other,Male,United-States
30507,43,14344,0,40,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,United-States
28911,32,0,0,40,Private,HS-grad,Married-civ-spouse,Transport-moving,Husband,White,Male,United-States
19484,39,0,0,30,Private,Bachelors,Married-civ-spouse,Sales,Wife,White,Female,United-States


In [14]:
model.predict(data_test)[:5]

array([' <=50K', ' <=50K', ' >50K', ' <=50K', ' >50K'], dtype=object)

In [15]:
target_test[:5]

7762      <=50K
23881     <=50K
30507      >50K
28911     <=50K
19484     <=50K
Name: class, dtype: object

To get directly the accuracy score, we need to call the `score` method. Let's
compute the accuracy score on the entire test set.

In [16]:
model.score(data_test, target_test)

0.8575874211776268

## Evaluation of the model with cross-validation

As previously stated, a predictive model should be evaluated by
cross-validation. Our model is usable with the cross-validation tools of
scikit-learn as any other predictors:

In [17]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv = 5)
cv_results

{'fit_time': array([2.08371925, 2.30369544, 2.09225845, 2.16638732, 2.09557629]),
 'score_time': array([0.10800147, 0.07232356, 0.07397676, 0.07422066, 0.07290554]),
 'test_score': array([0.8512642 , 0.8498311 , 0.84756347, 0.8523751 , 0.85513923])}

In [18]:
scores = cv_results['test_score']
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is: 0.851 +/- 0.003


The compound model has a higher predictive accuracy than the two models that
used numerical and categorical variables in isolation.

## Fitting a more powerful model

**Linear models** are nice because they are usually cheap to train,
**small** to deploy, **fast** to predict and give a **good baseline**.

However, it is often useful to check whether more complex models such as an
ensemble of decision trees can lead to higher predictive performance. In this
section we will use such a model called **gradient-boosting trees** and
evaluate its statistical performance. More precisely, the scikit-learn model
we will use is called `HistGradientBoostingClassifier`. Note that boosting
models will be covered in more details in a future module.

For tree-based models, the handling of numerical and categorical variables is
simpler than for linear models:
* we do **not need to scale the numerical features**
* using an **ordinal encoding for the categorical variables** is fine even if
  the encoding results in an arbitrary ordering

Therefore, for `HistGradientBoostingClassifier`, the preprocessing pipeline
is slightly simpler than the one we saw earlier for the `LogisticRegression`:

In [19]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

preprocessor = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_columns)],
    remainder="passthrough")

model = make_pipeline(preprocessor, HistGradientBoostingClassifier())

Now that we created our model, we can check its statistical performance.

In [20]:
%%time
_ = model.fit(data_train, target_train)

Wall time: 3.02 s


In [21]:
model.score(data_test, target_test)

0.8796986323806404

We can observe that we get significantly higher accuracies with the Gradient
Boosting model. This is often what we observe whenever the dataset has a
large number of samples and limited number of informative features (e.g. less
than 1000) with a mix of numerical and categorical variables.

This explains why Gradient Boosted Machines are very popular among
datascience practitioners who work with tabular data.

In this notebook we:

* used a `ColumnTransformer` to apply different preprocessing for
  categorical and numerical variables;
* used a pipeline to chain the `ColumnTransformer` preprocessing and
  logistic regression fitting;
* seen that **gradient boosting methods** can outperform **linear
  models**.